**Libraries**

In [1]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D,Activation, Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import Adam
from keras import optimizers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from keras import backend as K
#K.tensorflow_backend.set_session(sess)

**TL-models**

In [2]:
from keras.applications import resnet50, vgg16, vgg19, xception,densenet, inception_v3, mobilenet,  nasnet, inception_resnet_v2

**Add-ons**

In [3]:
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, EarlyStopping
from keras.applications.resnet50 import preprocess_input
import os
import datetime
import json
from keras.preprocessing.image import ImageDataGenerator

In [5]:
import glob
import cv2
from random import shuffle

In [6]:
bad_path = 'C:\\Users\\reekithak\\Deep Learning\\Images\\bad\\*.jpg'
good_path = 'C:\\Users\\reekithak\\Deep Learning\\Images\\good\\*.jpg'

In [7]:
addrs_b = glob.glob(bad_path)
addrs_g = glob.glob(good_path)
labels_b = [1 for addr in addrs_g]  # 1 = good, 0 =  bad
labels_g = [0 for addr in addrs_b]

**Good**

In [8]:
data_g = []
for imagePath in addrs_g:
    img = cv2.imread(imagePath)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    data_g.append(img)

**Bad**

In [9]:
data_b = []
for imagePath in addrs_b:
# load the image, pre-process it, and store it in the data list
    img = cv2.imread(imagePath)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    data_b.append(img)

**Shuffling**

In [10]:
shuffle_data = True
if shuffle_data:
    g = list(zip(data_g, labels_g))
    b = list(zip(data_b, labels_b))
    e = g + b
    shuffle(e)
    data, labels = zip(*e)
del data_g
del data_b
del addrs_b
del addrs_g
Y_train = np.array(labels)
X_train = np.array(data, dtype="int8")
X_train =  preprocess_input(X_train)

**Preprocess for Res-Net(50) Arch**

In [11]:
left_input = Input((224,224,3))
right_input = Input((224,224,3))
convnet = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

94773248/94765736 [==============================] - 1s 0us/step


In [12]:
# Add the final fully connected layers
x = convnet.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
preds = Dense(18, activation='sigmoid')(x) # Apply sigmoid
convnet = Model(inputs=convnet.input, outputs=preds)

In [13]:
#Applying above model for both the left and right images
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

In [14]:
Euc_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

In [15]:
# use and add the distance function
Euc_distance = Euc_layer([encoded_l, encoded_r])
#identify the prediction
prediction = Dense(1,activation='sigmoid')(Euc_distance)

In [16]:
#Define the network with the left and right inputs and the ouput prediction
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

In [17]:
optim = optimizers.SGD(lr=0.001, decay=.01, momentum=0.9, nesterov=True)

In [18]:
siamese_net.compile(loss="binary_crossentropy",optimizer=optim,metrics=["accuracy"])

In [19]:
siamese_net.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 18)           126367634   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 18)           0           model[0][0]                

**Creating Train and Test Data**

In [20]:
X_train.shape , Y_train.shape

((2130, 224, 224, 3), (2130,))

In [25]:
image_list = X_train[:180]
label_list = Y_train[:180]

In [26]:
left_input = []
right_input = []
targets = []
pairs = 8

In [27]:
#create the dataset to train on
for i in range(len(label_list)):
    for j in range(pairs):
# we need to make sure that we are not comparing with the same image
        compare_to = i
        while compare_to == i: 
            compare_to = random.randint(0,179)
        left_input.append(image_list[i])
        right_input.append(image_list[compare_to])
        if label_list[i] == label_list[compare_to]:
            # if the images are same then label - 1
            targets.append(1.)
        else:
            # if the images are different then label - 0
            targets.append(0.)

In [28]:
#removing single dim entries
left_input = np.squeeze(np.array(left_input))
right_input = np.squeeze(np.array(right_input))
targets = np.squeeze(np.array(targets))

In [29]:
# Creating test datasets - left, right images and target label
good_image = X_train[4] #good_image = 1, bad_image = 0
test_left = []
test_right = []
test_targets = []
for i in range(len(Y_train)-180):
    test_left.append(good_image)
    test_right.append(X_train[i+180])
    test_targets.append(Y_train[i+180])
test_left = np.squeeze(np.array(test_left))
test_right = np.squeeze(np.array(test_right))
test_targets = np.squeeze(np.array(test_targets))


In [ ]:
siamese_net.fit([left_input,right_input], targets,
          batch_size=16,
          epochs=30,
          verbose=1,
          validation_data=([test_left,test_right],test_targets))